<a href="https://colab.research.google.com/github/boshuaiYu/CaiCai_sklearn/blob/main/Data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 无量纲化：preprocessing.MinMaxScaler  数据归一化处理后符合正态分布

  只有feature_range参数,表示压缩的范围



In [ ]:
from sklearn.preprocessing import MinMaxScaler
data = [[-1,2],[-0.5,6],[0,10],[1,18]]

In [ ]:
import pandas as pd
pd.DataFrame(data)

,0,1
0,-1.0,2
1,-0.5,6
2,0.0,10
3,1.0,18


In [ ]:
scaler = MinMaxScaler()
scaler = scaler.fit(data)
result = scaler.transform(data)
result

array([[0.  , 0.  ],
       [0.25, 0.25],
       [0.5 , 0.5 ],
       [1.  , 1.  ]])

In [ ]:
result_ = scaler.fit_transform(data)
result_

array([[0.  , 0.  ],
       [0.25, 0.25],
       [0.5 , 0.5 ],
       [1.  , 1.  ]])

In [ ]:
scaler.inverse_transform(result) # 当数据被覆盖时可以将归一化数据进行逆转

array([[-1. ,  2. ],
       [-0.5,  6. ],
       [ 0. , 10. ],
       [ 1. , 18. ]])

In [ ]:
data = [[-1,2],[-0.5,6],[0,10],[1,18]]
scaler = MinMaxScaler(feature_range=[5,10])
result = scaler.fit_transform(data)
result

array([[ 5.  ,  5.  ],
       [ 6.25,  6.25],
       [ 7.5 ,  7.5 ],
       [10.  , 10.  ]])

当x的特征数据量比较大时，使用partial_fit作为训练接口
scaler = scaler.partial_fit(data)

例：使用numpy实现MinMaxScaler函数

只是对不同的数据进行归一化，*而不是对特征进行归一化*

所以只会对列进行归一化

In [ ]:
import numpy as np
data = np.array([[-1,2],[-0.5,6],[0,10],[1,18]])
pd.DataFrame(data)

,0,1
0,-1.0,2.0
1,-0.5,6.0
2,0.0,10.0
3,1.0,18.0


In [ ]:
data.min()  # 返回的是所以值中的最小值

-1.0

In [ ]:
data.min(axis=0),data.min(axis=1)  # 分别返回的是每一列，每一行的最小值

(array([-1.,  2.]), array([-1. , -0.5,  0. ,  1. ]))

In [ ]:
def MinMaxScaler1(data):
  return (data - data.min(axis=0))/(data.max(axis=0)-data.min(axis=0))

def reverse(data):
  return MinMaxScaler1(data) * (data.max(axis=0)-data.min(axis=0)) + data.min(axis=0)

data = np.array([[-1,2],[-0.5,6],[0,10],[1,18]])
print(MinMaxScaler1(data))
print(reverse(data))

[[0.   0.  ]
 [0.25 0.25]
 [0.5  0.5 ]
 [1.   1.  ]]
[[-1.   2. ]
 [-0.5  6. ]
 [ 0.  10. ]
 [ 1.  18. ]]


2. 标准化：preprocessing.StandardScaler

  数据按照均值中心化，按标准差进行缩放，数据服从均值0，方差为1的正态方程

In [ ]:
from sklearn.preprocessing import StandardScaler

data = [[-1,2],[-0.5,6],[0,10],[1,18]]

StandardScaler()

In [ ]:
scaler = StandardScaler()
scaler.fit(data)

In [ ]:
scaler.mean_ # 自动按列进行计算
# 计算均值

array([-0.125,  9.   ])

In [ ]:
scaler.var_
# 计算方差

array([ 0.546875, 35.      ])

In [ ]:
x_std = scaler.transform(data)
x_std

array([[-1.18321596, -1.18321596],
       [-0.50709255, -0.50709255],
       [ 0.16903085,  0.16903085],
       [ 1.52127766,  1.52127766]])

In [ ]:
x_std.mean()

0.0

In [ ]:
x_std.std()

1.0

In [ ]:
scaler.fit_transform(data)

array([[-1.18321596, -1.18321596],
       [-0.50709255, -0.50709255],
       [ 0.16903085,  0.16903085],
       [ 1.52127766,  1.52127766]])

In [ ]:
scaler.inverse_transform(x_std)

array([[-1. ,  2. ],
       [-0.5,  6. ],
       [ 0. , 10. ],
       [ 1. , 18. ]])

空值NaN会被当做是缺失值，在fit的时候忽略，在transform的时候保持缺失NaN的状态显示。并且，尽管去量纲化过程不是具体的算法，但在fit接口中，依然只允许导入至少二维数组，一维数组导入会报错。通常来说，我们输入的X会是我们的特征矩阵，现实案例中特征矩阵不太可能是一维所以不会存在这个问题

3. 缺失值处理

In [ ]:
import pandas as pd
data = pd.read_csv("/content/Narrativedata.csv",index_col=0) # 第一列等于索引
data.head()

,Age,Sex,Embarked,Survived
0,22.0,male,S,No
1,38.0,female,C,Yes
2,26.0,female,S,Yes
3,35.0,female,S,Yes
4,35.0,male,S,No


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       714 non-null    float64
 1   Sex       891 non-null    object 
 2   Embarked  889 non-null    object 
 3   Survived  891 non-null    object 
dtypes: float64(1), object(3)
memory usage: 34.8+ KB


impute.SimpleImputer类
1. missing_values
2. strategy(
  mean使用均值填补
  median用中值填补(仅对数值型特征有效)
  most_frequent用众数填补 
  constant"表示请参考参数fill_value值(对数值型和类别型都适用）
3. fill_value(当参数startegy为”constant"的时候可用，可输入字符串或数字表示要填充的值，常用0)
4. copy(创建副本)

In [ ]:
Age = data["Age"] # data.loc[:,"Age"]等价
Age = Age.values.reshape(-1,1) # sklearn中的特征矩阵必须是二维的
Age.shape

(891, 1)

In [ ]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer()
imp_median = SimpleImputer(strategy="median")
imp_0 = SimpleImputer(strategy="constant",fill_value=0)

In [ ]:
Age[:20]

array([[22.],
       [38.],
       [26.],
       [35.],
       [35.],
       [nan],
       [54.],
       [ 2.],
       [27.],
       [14.],
       [ 4.],
       [58.],
       [20.],
       [39.],
       [14.],
       [55.],
       [ 2.],
       [nan],
       [31.],
       [nan]])

In [ ]:
imp_mean = imp_mean.fit_transform(Age)
imp_median = imp_median.fit_transform(Age)
imp_0 = imp_0.fit_transform(Age)

In [ ]:
imp_mean[:20]

array([[22.        ],
       [38.        ],
       [26.        ],
       [35.        ],
       [35.        ],
       [29.69911765],
       [54.        ],
       [ 2.        ],
       [27.        ],
       [14.        ],
       [ 4.        ],
       [58.        ],
       [20.        ],
       [39.        ],
       [14.        ],
       [55.        ],
       [ 2.        ],
       [29.69911765],
       [31.        ],
       [29.69911765]])

In [ ]:
imp_median[:20]

array([[22.],
       [38.],
       [26.],
       [35.],
       [35.],
       [28.],
       [54.],
       [ 2.],
       [27.],
       [14.],
       [ 4.],
       [58.],
       [20.],
       [39.],
       [14.],
       [55.],
       [ 2.],
       [28.],
       [31.],
       [28.]])

In [ ]:
imp_0[:20]

array([[22.],
       [38.],
       [26.],
       [35.],
       [35.],
       [ 0.],
       [54.],
       [ 2.],
       [27.],
       [14.],
       [ 4.],
       [58.],
       [20.],
       [39.],
       [14.],
       [55.],
       [ 2.],
       [ 0.],
       [31.],
       [ 0.]])

In [ ]:
data.loc[:,"Age"] = imp_median
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       891 non-null    float64
 1   Sex       891 non-null    object 
 2   Embarked  889 non-null    object 
 3   Survived  891 non-null    object 
dtypes: float64(1), object(3)
memory usage: 34.8+ KB


In [ ]:
Embarked = data["Embarked"].values.reshape(-1,1)

In [ ]:
imp_mode = SimpleImputer(strategy="most_frequent")
data["Embarked"] = imp_mode.fit_transform(Embarked)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       891 non-null    float64
 1   Sex       891 non-null    object 
 2   Embarked  891 non-null    object 
 3   Survived  891 non-null    object 
dtypes: float64(1), object(3)
memory usage: 34.8+ KB


使用pandas和numpy进行填充

In [ ]:
data1 = pd.read_csv("/content/Narrativedata.csv",index_col=0)
data1.head(10)

,Age,Sex,Embarked,Survived
0,22.0,male,S,No
1,38.0,female,C,Yes
2,26.0,female,S,Yes
3,35.0,female,S,Yes
4,35.0,male,S,No
5,NaN,male,Q,No
6,54.0,male,S,No
7,2.0,male,S,No
8,27.0,female,S,Yes
9,14.0,female,C,Yes


In [ ]:
data1["Age"] = data1["Age"].fillna(data["Age"].median()) # fillna可以在DataFrame里面直接填充
data1.head(10)

,Age,Sex,Embarked,Survived
0,22.0,male,S,No
1,38.0,female,C,Yes
2,26.0,female,S,Yes
3,35.0,female,S,Yes
4,35.0,male,S,No
5,28.0,male,Q,No
6,54.0,male,S,No
7,2.0,male,S,No
8,27.0,female,S,Yes
9,14.0,female,C,Yes


In [ ]:
data1.dropna(axis=0,inplace=True)
# .dropna(axis=0)删除所有有缺失值的行，.dropna(axis=1)删除所有有缺失值的列
# 参数inplace，为True表示在原数据集上进行修改，为False表示生成一个复制对象，不修改原数据，默认False

In [ ]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       889 non-null    float64
 1   Sex       889 non-null    object 
 2   Embarked  889 non-null    object 
 3   Survived  889 non-null    object 
dtypes: float64(1), object(3)
memory usage: 34.7+ KB


4. 处理分类型特征：编码及哑变量

preprocessing.LabelEncoder: 标签专用，能够将分类转换成分类数值


In [ ]:
from sklearn.preprocessing import LabelEncoder

y = data.iloc[:,-1] # 输入的是标签，不是矩阵，允许一维

In [ ]:
le = LabelEncoder()
le = le.fit(y)
label = le.transform(y)

In [ ]:
le.classes_  # 共三类

array(['No', 'Unknown', 'Yes'], dtype=object)

In [ ]:
label

In [ ]:
data.iloc[:,-1] = label


In [ ]:
data.head(10)

,Age,Sex,Embarked,Survived
0,22.0,male,S,0
1,38.0,female,C,2
2,26.0,female,S,2
3,35.0,female,S,2
4,35.0,male,S,0
5,28.0,male,Q,0
6,54.0,male,S,0
7,2.0,male,S,0
8,27.0,female,S,2
9,14.0,female,C,2


In [ ]:
# 综合起来
from sklearn.preprocessing import LabelEncoder
data.iloc[:,-1] = LabelEncoder().fit_transform(data.iloc[:,-1])
data.head() # 与上述结果相同

,Age,Sex,Embarked,Survived
0,22.0,male,S,0
1,38.0,female,C,2
2,26.0,female,S,2
3,35.0,female,S,2
4,35.0,male,S,0


preprocessing.OrdinalEncoder: 特征专用，能够将分类特征转换为分类数值

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
data_ = data.copy()
data_.head()

,Age,Sex,Embarked,Survived
0,22.0,male,S,0
1,38.0,female,C,2
2,26.0,female,S,2
3,35.0,female,S,2
4,35.0,male,S,0


In [ ]:
OrdinalEncoder().fit(data_.iloc[:,1:-1]).categories_

[array(['female', 'male'], dtype=object), array(['C', 'Q', 'S'], dtype=object)]

In [ ]:
data_.iloc[:,1:-1] = OrdinalEncoder().fit_transform(data_.iloc[:,1:-1])

In [ ]:
data_.head()

,Age,Sex,Embarked,Survived
0,22.0,1.0,2.0,0
1,38.0,0.0,0.0,2
2,26.0,0.0,2.0,2
3,35.0,0.0,2.0,2
4,35.0,1.0,2.0,0


precessing.OneHotEncoder: 独热编码，创建哑变量

In [ ]:
data.head()

,Age,Sex,Embarked,Survived
0,22.0,male,S,0
1,38.0,female,C,2
2,26.0,female,S,2
3,35.0,female,S,2
4,35.0,male,S,0


In [ ]:
from sklearn.preprocessing import OneHotEncoder
X = data.iloc[:,1:-1]

In [ ]:
enc = OneHotEncoder(categories="auto").fit(X)
result = enc.transform(X).toarray()

In [ ]:
enc.transform(X)

<891x5 sparse matrix of type '<class 'numpy.float64'>'
	with 1782 stored elements in Compressed Sparse Row format>

In [ ]:
result

array([[0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0.]])

In [ ]:
pd.DataFrame(enc.inverse_transform(result))

,0,1
0,male,S
1,female,C
2,female,S
3,female,S
4,male,S
...,...,...
886,male,S
887,female,S
888,female,S
889,male,C


In [ ]:
enc.get_feature_names() # 每个哑变量指的是哪个类型

array(['x0_female', 'x0_male', 'x1_C', 'x1_Q', 'x1_S'], dtype=object)

In [ ]:
data.head()

,Age,Sex,Embarked,Survived
0,22.0,male,S,0
1,38.0,female,C,2
2,26.0,female,S,2
3,35.0,female,S,2
4,35.0,male,S,0


In [ ]:
newdata = pd.concat([data,pd.DataFrame(result)],axis=1)
newdata.head()

,Age,Sex,Embarked,Survived,0,1,2,3,4
0,22.0,male,S,0,0.0,1.0,0.0,0.0,1.0
1,38.0,female,C,2,1.0,0.0,1.0,0.0,0.0
2,26.0,female,S,2,1.0,0.0,0.0,0.0,1.0
3,35.0,female,S,2,1.0,0.0,0.0,0.0,1.0
4,35.0,male,S,0,0.0,1.0,0.0,0.0,1.0


In [ ]:
newdata.drop(["Sex","Embarked"],axis=1,inplace=True)
# axis=1,表示跨行进行合并，也就是将量表左右相连，如果是axis=0，就是将量表上下相连
newdata.columns = ["Age","Survived","Female","Male","Embarked_C","Embarked_D","Embarked_E"]
newdata.head()

,Age,Survived,Female,Male,Embarked_C,Embarked_D,Embarked_E
0,22.0,0,0.0,1.0,0.0,0.0,1.0
1,38.0,2,1.0,0.0,1.0,0.0,0.0
2,26.0,2,1.0,0.0,0.0,0.0,1.0
3,35.0,2,1.0,0.0,0.0,0.0,1.0
4,35.0,0,0.0,1.0,0.0,0.0,1.0


5. 处理连续型特征：二值化与分段

In [80]:
data_2 = data.copy()
from sklearn.preprocessing import Binarizer


In [ ]:
X = data.iloc[:,0].values.reshape(-1,1)
transformer = Binarizer(threshold=30).fit_transform(X)
data.iloc[:,0] = transformer

preprocessing.KBinsDiscretizer
这是将连续型变量划分为分类变量的类，能够将连续型变量排序后按顺序分箱后编码

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
x = data.iloc[:,0].values.reshape(-1,1)
est = KBinsDiscretizer(n_bins=3, encode="ordinal",strategy="uniform")
est.fit_transform(X)

In [92]:
set(est.fit_transform(X).ravel()) # 查看一列中的三箱

{0.0, 1.0, 2.0}

In [93]:
est = KBinsDiscretizer(n_bins=3, encode="onehot",strategy="uniform")
est.fit_transform(X).toarray()

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])